In [1]:
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

# Dados observatório educacional

## Carregar os dados

In [2]:
df = pd.read_csv("../output/data.csv", low_memory=False)

df.head()

,NO_MUNICIPIO,ANO,TP_DEPENDENCIA,TP_CATEGORIA_ESCOLA_PRIVADA,TP_LOCALIZACAO,TP_LOCALIZACAO_DIFERENCIADA,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,...,QT_MAT_INF_PRE_INT,QT_DOC_INF,QT_DOC_INF_CRE,QT_DOC_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,IN_PROF_GESTAO,IN_PROF_COORDENADOR,QT_PROF_COORDENADOR
0,Almadina,2015,Municipal,Não aplicável para escolas públicas,Urbana,Escola não está em área de localização diferen...,Sim,Próprio,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
1,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
2,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
3,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Não,Não aplicável para escolas que não ocupam préd...,Não,Não,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável
4,Almadina,2015,Municipal,Não aplicável para escolas públicas,Rural,Escola não está em área de localização diferen...,Sim,Cedido,Não,Sim,...,0,0,0,0,0,0,0,Não aplicável,Não aplicável,Não aplicável


## Funções úteis

In [3]:
def fill_NaN_and_Infs(df: pd.DataFrame, column: str) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy[column] = df_copy[column].apply(lambda x: 0 if np.isnan(x) or np.isinf(x) else x)
    return df_copy

def fill_NaN_and_Infs_all(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    for column in df_copy.columns:
        df_copy = fill_NaN_and_Infs(df_copy, column)
    return df_copy

def replace_nao_aplicavel(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    df_copy = df.copy()
    for column in columns:
        df_copy[column] = df_copy[column].replace("Não aplicável", 0)
        df_copy[column] = df_copy[column].astype(int)
    return df_copy

def get_mean_columns(df: pd.DataFrame, first_column: str, second_column: str) -> pd.Series:
    df_copy = df[[first_column, second_column]].copy()
    df_copy['MDA'] = df_copy[first_column] / df_copy[second_column]
    df_copy = fill_NaN_and_Infs(df_copy, 'MDA')
    return df_copy['MDA']

def get_mean_columns_pivoted(df: pd.DataFrame, mean_columns: list[str], pivot_column: str,
                             ) -> pd.Series:
    df_copy = df[['NO_MUNICIPIO', 'ANO', pivot_column] + mean_columns].copy()
    df_copy = replace_nao_aplicavel(df_copy, mean_columns)
    df_copy = df_copy.groupby(by=["NO_MUNICIPIO", "ANO", pivot_column],
                              as_index=False).sum(numeric_only=True)
    df_copy['MDA'] = get_mean_columns(df_copy, mean_columns[0], mean_columns[1])
    df_copy = df_copy.pivot_table(index=['NO_MUNICIPIO', 'ANO'],
                                  columns=pivot_column, values='MDA',
                    aggfunc='sum', fill_value=0).reset_index()

    return df_copy

def replace_sim_nao_to_binary(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    df_copy = df.copy()
    for column in columns:
        df_copy[column] = df_copy[column].replace(r'\bSim\b', 1, regex=True)
        df_copy[column] = df_copy[column].replace(r'\bNão\b', 0, regex=True)
        df_copy[column] = df_copy[column].astype(int)
    return df_copy

def get_prop_columns(df: pd.DataFrame, first_column: str, second_column: str) -> pd.Series:
    df_copy = df[[first_column, second_column]].copy()
    df_copy['PROP'] = df_copy[first_column] / df_copy[second_column]
    df_copy = fill_NaN_and_Infs(df_copy, 'PROP')
    df_copy['PROP'] = df_copy['PROP'].apply(lambda x: 0 if x > 1 else x)
    df_copy['PROP'] = df_copy['PROP'].round(2)
    return df_copy['PROP']

## Dados mais específicos da educação infantil dos municípios

In [4]:
columns = ['QT_MAT_BAS_0_3', 'QT_MAT_BAS_4_5', 'QT_MAT_INF', 'QT_DOC_INF']

df_especifico = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_especifico = replace_nao_aplicavel(df_especifico, columns)
df_especifico['QT_ESC'] = df_especifico['QT_MAT_INF'].apply(lambda x: 1 if x > 0 else 0)

df_especifico = df_especifico.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_especifico

,NO_MUNICIPIO,ANO,QT_MAT_BAS_0_3,QT_MAT_BAS_4_5,QT_MAT_INF,QT_DOC_INF,QT_ESC
0,Almadina,2015,51,152,207,19,3
1,Almadina,2016,56,128,189,17,3
2,Almadina,2017,65,127,207,14,4
3,Almadina,2018,93,130,233,16,5
4,Almadina,2019,92,134,235,14,5
...,...,...,...,...,...,...,...
220,Una,2019,229,509,820,80,26
221,Una,2020,235,472,745,68,23
222,Una,2021,212,465,726,60,20
223,Una,2022,243,472,774,67,22


__Dados extraídos__:

- QT_MAT_BAS_0_3: População de 0 a 3 de idade escolar
- QT_MAT_BAS_4_5: População de 4 a 5 de idade escolar
- QT_MAT_INF: Número geral de matrículas
- QT_DOC_INF: Número geral de docentes
- QT_ESC: Número geral de escolas

## Média de alunos por turma na pré-escola e creche

In [5]:
# Média de alunos por turma
columns = ['QT_MAT_INF', 'QT_MAT_INF_CRE', 'QT_MAT_INF_PRE', 'QT_TUR_INF',
           'QT_TUR_INF_CRE', 'QT_TUR_INF_PRE']

df_mda_alu_tur = df[['NO_MUNICIPIO', 'ANO'] + columns].copy()
df_mda_alu_tur = replace_nao_aplicavel(df_mda_alu_tur, columns)
df_mda_alu_tur['MDA_ALU_TUR_INF'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF', 'QT_TUR_INF')
df_mda_alu_tur['MDA_ALU_TUR_CRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_CRE', 'QT_TUR_INF_CRE')
df_mda_alu_tur['MDA_ALU_TUR_PRE'] = get_mean_columns(df_mda_alu_tur, 'QT_MAT_INF_PRE', 'QT_TUR_INF_PRE')

df_mda_alu_tur = df_mda_alu_tur.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(numeric_only=True)
df_mda_alu_tur

# Média de alunos por turma segundo dependência administrativa
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp = get_mean_columns_pivoted(df, mean_columns, 'TP_DEPENDENCIA')
tmp.rename(columns={'Estadual': 'MDA_ALU_TUR_INF_EST', 'Federal': 'MDA_ALU_TUR_INF_FED',
                     'Municipal': 'MDA_ALU_TUR_INF_MUN', 'Privada': 'MDA_ALU_TUR_INF_PRI'}, inplace=True)

# Média de alunos por turma segundo localização
mean_columns = ['QT_MAT_INF', 'QT_TUR_INF']
tmp2 = get_mean_columns_pivoted(df, mean_columns, 'TP_LOCALIZACAO')
tmp2.rename(columns={'Urbana': 'MDA_ALU_TUR_INF_URB', 'Rural': 'MDA_ALU_TUR_INF_RUR'}, inplace=True)

df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur = pd.merge(df_mda_alu_tur, tmp2, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_mda_alu_tur

,NO_MUNICIPIO,ANO,QT_MAT_INF,QT_MAT_INF_CRE,QT_MAT_INF_PRE,QT_TUR_INF,QT_TUR_INF_CRE,QT_TUR_INF_PRE,MDA_ALU_TUR_INF,MDA_ALU_TUR_CRE,MDA_ALU_TUR_PRE,MDA_ALU_TUR_INF_EST,MDA_ALU_TUR_INF_FED,MDA_ALU_TUR_INF_MUN,MDA_ALU_TUR_INF_PRI,MDA_ALU_TUR_INF_RUR,MDA_ALU_TUR_INF_URB
0,Almadina,2015,207,48,159,10,2,8,55.750000,24.000000,56.000000,0.0,0.0,23.875000,8.000000,0.000000,20.700000
1,Almadina,2016,189,53,136,9,2,7,56.250000,26.500000,50.250000,0.0,0.0,22.125000,12.000000,0.000000,21.000000
2,Almadina,2017,207,69,138,8,2,6,49.500000,33.500000,44.250000,0.0,0.0,25.875000,0.000000,0.000000,24.750000
3,Almadina,2018,233,102,131,12,4,8,52.650000,43.333333,45.750000,0.0,0.0,22.888889,9.000000,0.000000,19.000000
4,Almadina,2019,235,92,143,11,2,9,59.750000,46.000000,47.500000,0.0,0.0,26.750000,7.000000,0.000000,21.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Una,2019,820,231,589,56,11,45,280.608333,33.375000,247.566667,0.0,0.0,16.702128,3.888889,11.541667,16.968750
221,Una,2020,745,276,469,50,12,38,275.375000,75.375000,228.333333,0.0,0.0,17.119048,3.250000,12.900000,16.233333
222,Una,2021,726,226,500,43,8,35,265.500000,26.500000,227.000000,0.0,0.0,17.609756,2.000000,13.315789,19.708333
223,Una,2022,774,265,509,53,13,40,256.638889,30.222222,213.083333,0.0,0.0,16.954545,3.111111,11.095238,16.906250


__Dados extraídos__:

- QT_MAT_INF: Número geral de matrículas
- QT_MAT_INF_CRE: Número de matrículas em creches
- QT_MAT_INF_PRE: Número de matrículas em pré-escolas
- QT_TUR_INF: Número geral de turmas
- QT_TUR_INF_CRE: Número de turmas em creches
- QT_TUR_INF_PRE: Número de turmas em pré-escolas
- MDA_ALU_TUR_INF: Média geral de alunos por turma
- MDA_ALU_TUR_INF_CRE: Média de alunos por turma em creches
- MDA_ALU_TUR_INF_PRE: Média de alunos por turma em pré-escolas
- MDA_ALU_TUR_INF_EST: Média de alunos por turma em escolas estaduais
- MDA_ALU_TUR_INF_FED: Média de alunos por turma em escolas federais
- MDA_ALU_TUR_INF_MUN: Média de alunos por turma em escolas municipais
- MDA_ALU_TUR_INF_PRI: Média de alunos por turma em escolas privadas
- MDA_ALU_TUR_INF_RUR: Média de alunos por turma em localidades rurais
- MDA_ALU_TUR_INF_URB: Média de alunos por turma em localidades urbanas

## Número de estabelecimentos de educação infantil (creches)

In [38]:
columns_to_replace = [
    "IN_AGUA_INEXISTENTE",
    "IN_ESGOTO_INEXISTENTE",
    "IN_ENERGIA_INEXISTENTE",
    "IN_AGUA_POTAVEL",
    "IN_BANHEIRO_EI",
    "IN_PARQUE_INFANTIL",
]
columns = ["QT_MAT_INF_CRE"] + columns_to_replace

df_estab_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()

df_estab_cre = replace_sim_nao_to_binary(df_estab_cre, columns_to_replace)
df_estab_cre = replace_nao_aplicavel(df_estab_cre, columns)
df_estab_cre["QT_CRE"] = df_estab_cre["QT_MAT_INF_CRE"].apply(
    lambda x: 1 if x > 0 else 0
)

df_estab_cre = df_estab_cre.groupby(by=["NO_MUNICIPIO", "ANO"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_CRE")
df_estab_cre["PROP_ESGOTO_INEXISTENTE_CRE"] = get_prop_columns(
    df_estab_cre, "IN_ESGOTO_INEXISTENTE", "QT_CRE"
)
df_estab_cre["PROP_ENERGIA_INEXISTENTE_CRE"] = get_prop_columns(
    df_estab_cre, "IN_ENERGIA_INEXISTENTE", "QT_CRE"
)
df_estab_cre["PROP_AGUA_INEXISTENTE_CRE"] = get_prop_columns(
    df_estab_cre, "IN_AGUA_INEXISTENTE", "QT_CRE"
)
df_estab_cre["PROP_AGUA_POTAVEL_CRE"] = get_prop_columns(
    df_estab_cre, "IN_AGUA_POTAVEL", "QT_CRE"
)
df_estab_cre["PROP_BANHEIRO_EI_CRE"] = get_prop_columns(
    df_estab_cre, "IN_BANHEIRO_EI", "QT_CRE"
)
df_estab_cre["PROP_PARQUE_EI_CRE"] = get_prop_columns(
    df_estab_cre, "IN_PARQUE_INFANTIL", "QT_CRE"
)
df_estab_cre = df_estab_cre.drop(columns=columns_to_replace)
df_estab_cre

columns = ['QT_MAT_INF_CRE']
tmp =  df[["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"] + columns].copy()
tmp = replace_nao_aplicavel(tmp, columns)
tmp["QT_CRE"] = tmp["QT_MAT_INF_CRE"].apply(
    lambda x: 1 if x > 0 else 0
)
tmp = tmp.groupby(by=["NO_MUNICIPIO", "ANO", "TP_DEPENDENCIA"], as_index=False).sum(
    numeric_only=True
).drop(columns="QT_MAT_INF_CRE")
tmp = tmp.pivot_table(index=["NO_MUNICIPIO", "ANO"], columns="TP_DEPENDENCIA", values='QT_CRE',
                aggfunc='sum', fill_value=0).reset_index()
tmp.rename(columns={'Estadual': 'QT_CRE_EST', 'Federal': 'QT_CRE_FED',
                        'Municipal': 'QT_CRE_MUN', 'Privada': 'QT_CRE_PRI'}, inplace=True)

df_estab_cre = pd.merge(df_estab_cre, tmp, on=['NO_MUNICIPIO', 'ANO'], how='left')
df_estab_cre

,NO_MUNICIPIO,ANO,QT_CRE,PROP_ESGOTO_INEXISTENTE_CRE,PROP_ENERGIA_INEXISTENTE_CRE,PROP_AGUA_INEXISTENTE_CRE,PROP_AGUA_POTAVEL_CRE,PROP_BANHEIRO_EI_CRE,PROP_PARQUE_EI_CRE,QT_CRE_EST,QT_CRE_FED,QT_CRE_MUN,QT_CRE_PRI
0,Almadina,2015,1,1.0,1.00,0.0,0.0,0.00,0.00,0,0,1,0
1,Almadina,2016,1,1.0,0.00,0.0,0.0,0.00,0.00,0,0,1,0
2,Almadina,2017,2,0.5,0.00,0.0,0.0,0.50,0.00,0,0,2,0
3,Almadina,2018,2,0.5,0.00,0.0,0.0,1.00,0.00,0,0,1,1
4,Almadina,2019,1,1.0,0.00,0.0,0.0,0.00,0.00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,Una,2019,6,0.0,0.33,0.0,0.0,0.67,0.17,0,0,3,3
221,Una,2020,10,0.0,0.00,0.0,0.0,0.40,0.10,0,0,8,2
222,Una,2021,6,0.0,0.00,0.0,0.0,0.67,0.17,0,0,6,0
223,Una,2022,8,0.0,0.00,0.0,0.0,0.62,0.12,0,0,6,2


__Dados extraídos__:

- QT_CRE: Número geral de matrículas
- PROP_ESGOTO_INEXISTENTE_CRE: Proporção de creches sem qualquer forma de coleta de esgoto
- PROP_ENERGIA_INEXISTENTE_CRE: Proporção de creches sem qualquer forma de acesso à energia elétrica
- PROP_AGUA_INEXISTENTE_CRE: Proporção de creches sem qualquer forma de distribuição de água
- PROP_AGUA_POTAVEL_CRE: Proporção de creches que oferecem água potável 
- PROP_BANHEIRO_EI_CRE: Proporção de creches que possuem banheiro adequado à educação infantil
- PROP_PARQUE_EI_CRE: Proporção de creches que possuem parque infantil
- QT_CRE_EST: Número de creches estaduais
- QT_CRE_FED: Número de creches federais
- QT_CRE_MUN: Número de creches municipais
- QT_CRE_PRI: Número de creches privadas

## Número de estabelecimentos de educação infantil (pré-escola)

In [7]:
columns_to_replace = [
    "IN_ESGOTO_INEXISTENTE",
    "IN_ENERGIA_INEXISTENTE",
    "IN_AGUA_POTAVEL",
]
columns = ["QT_MAT_INF_PRE"] + columns_to_replace

df_estab_pre = df[["NO_MUNICIPIO", "TP_LOCALIZACAO", "ANO"] + columns].copy()

df_estab_pre = replace_sim_nao_to_binary(df_estab_pre, columns_to_replace)
df_estab_pre = replace_nao_aplicavel(df_estab_pre, columns)
df_estab_pre["QT_PRE"] = df_estab_pre["QT_MAT_INF_PRE"].apply(
    lambda x: 1 if x > 0 else 0
)

df_estab_pre = df_estab_pre.groupby(by=["NO_MUNICIPIO", "TP_LOCALIZACAO", "ANO"], as_index=False).sum(
    numeric_only=True
)
df_estab_pre["PROP_ENERGIA_INEXISTENTE_PRE"] = get_prop_columns(
    df_estab_pre, "IN_ENERGIA_INEXISTENTE", "QT_PRE"
)
df_estab_pre["PROP_ESGOTO_INEXISTENTE_PRE"] = get_prop_columns(
    df_estab_pre, "IN_ESGOTO_INEXISTENTE", "QT_PRE"
)
df_estab_pre["PROP_AGUA_POTAVEL_PRE"] = get_prop_columns(
    df_estab_pre, "IN_AGUA_POTAVEL", "QT_PRE"
)
df_estab_pre = df_estab_pre.drop(columns=columns_to_replace)
df_estab_pre

,NO_MUNICIPIO,TP_LOCALIZACAO,ANO,QT_MAT_INF_PRE,QT_PRE,PROP_ENERGIA_INEXISTENTE_PRE,PROP_ESGOTO_INEXISTENTE_PRE,PROP_AGUA_POTAVEL_PRE
0,Almadina,Rural,2015,0,0,0.0,0.0,0.0
1,Almadina,Rural,2016,0,0,0.0,0.0,0.0
2,Almadina,Rural,2017,7,2,0.0,0.5,0.0
3,Almadina,Rural,2018,5,2,0.0,0.5,0.0
4,Almadina,Rural,2019,3,2,0.0,0.5,0.0
...,...,...,...,...,...,...,...,...
445,Una,Urbana,2019,325,7,0.0,0.0,0.0
446,Una,Urbana,2020,269,7,0.0,0.0,0.0
447,Una,Urbana,2021,261,5,0.0,0.0,0.0
448,Una,Urbana,2022,299,6,0.0,0.0,0.0


## NÚMERO DE MATRÍCULAS EM CRECHES

In [8]:
columns_to_replace = [
    "TP_DEPENDENCIA",
]
columns = ["QT_MAT_INF_CRE"] + columns_to_replace

df_matr_cre = df[["NO_MUNICIPIO", "ANO"] + columns].copy()

df_matr_cre = replace_sim_nao_to_binary(df_matr_cre, columns_to_replace)
df_matr_cre = replace_nao_aplicavel(df_matr_cre, columns)
df_matr_cre["QT_CRE"] = df_matr_cre["QT_MAT_INF_CRE"].apply(
    lambda x: 1 if x > 0 else 0
)

df_matr_cre = df_matr_cre.drop(columns=columns_to_replace)
df_matr_cre

ValueError: invalid literal for int() with base 10: 'Municipal'

## NÚMERO DE MATRÍCULAS EM PRÉ-ESCOLAS